# The Price is Right

In this notebook I build a more complex solution for estimating prices of goods.

1. Part 2a: creates a RAG database with my 400,000 training data
2. Part 2b: visualize in 2D
3. Part 2c: visualize in 3D
4. Part 2d: build and test a RAG pipeline with GPT-4o-mini
5. Part 2e: (a) build a Random Forest pricer (b) Create a Ensemble pricer that allows contributions from all the pricers

In [1]:
# imports

import os
import re
import math
import json
from tqdm import tqdm
import random
from dotenv import load_dotenv
from huggingface_hub import login
import numpy as np
import pickle
from sentence_transformers import SentenceTransformer
from datasets import load_dataset
import chromadb
from sklearn.manifold import TSNE
import plotly.graph_objects as go

In [2]:
# environment

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')
DB = "products_vectorstore"

In [3]:
# Log in to HuggingFace

hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [4]:
# Another import after Logging in to Hugging Face - thank you Trung N.!

from items import Item

## Load the dataset

These were created in notebooks:
1. [Part-1-Reading-cleaning-data.ipynb](https://github.com/shreyagupta/deep-llm-engineering/blob/main/Agentic%20and%20other%20examples%20using%20LLM%20APIs/curating_data_to_finetune_or_directly_use_frontier_models/Part-1-Reading-cleaning-data.ipynb)
2. [Part-2-pre-processing-data-for-train-test.ipynb](https://github.com/shreyagupta/deep-llm-engineering/blob/main/Agentic%20and%20other%20examples%20using%20LLM%20APIs/curating_data_to_finetune_or_directly_use_frontier_models/Part-2-pre-processing-data-for-train-test.ipynb)

In [7]:
# With train.pkl in this folder, you can run this:

with open('train.pkl', 'rb') as file:
    train = pickle.load(file)

In [8]:
train[0].prompt

'How much does this cost to the nearest dollar?\n\nDelphi FG0166 Fuel Pump Module\nDelphi brings 80 years of OE Heritage into each Delphi pump, ensuring quality and fitment for each Delphi part. Part is validated, tested and matched to the right vehicle application Delphi brings 80 years of OE Heritage into each Delphi assembly, ensuring quality and fitment for each Delphi part Always be sure to check and clean fuel tank to avoid unnecessary returns Rigorous OE-testing ensures the pump can withstand extreme temperatures Brand Delphi, Fit Type Vehicle Specific Fit, Dimensions LxWxH 19.7 x 7.7 x 5.1 inches, Weight 2.2 Pounds, Auto Part Position Unknown, Operation Mode Mechanical, Manufacturer Delphi, Model FUEL PUMP, Dimensions 19.7\n\nPrice is $227.00'

# Now I create a Chroma Datastore

In [9]:
client = chromadb.PersistentClient(path=DB)

In [10]:
# Check if the collection exists and delete it if it does
collection_name = "products"

# For old versions of Chroma, use this line instead of the subsequent one
# existing_collection_names = [collection.name for collection in client.list_collections()]
existing_collection_names = client.list_collections()

if collection_name in existing_collection_names:
    client.delete_collection(collection_name)
    print(f"Deleted existing collection: {collection_name}")

collection = client.create_collection(collection_name)

# Introducing the SentenceTransfomer

The all-MiniLM is a very useful model from HuggingFace that maps sentences & paragraphs to a 384 dimensional dense vector space and is ideal for tasks like semantic search.

https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

It can run pretty quickly locally.

Benefits of using all-MiniLM as compared to OpenAI embeddings:
1. It's free and fast!
3. I can run it locally, so the data never leaves my box - might be useful if you're building a personal RAG

In [11]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [12]:
# Pass in a list of texts, get back a numpy array of vectors

vector = model.encode(["Well hi there"])[0]
vector

array([-9.46716368e-02,  4.27618697e-02,  5.51619120e-02, -5.11071295e-04,
        1.16204470e-02, -6.80130646e-02,  2.76404843e-02,  6.06974922e-02,
        2.88530756e-02, -1.74127687e-02, -4.94346619e-02,  2.30993833e-02,
       -1.28614632e-02, -4.31402586e-02,  2.17510499e-02,  4.26548757e-02,
        5.10500371e-02, -7.79727921e-02, -1.23247311e-01,  3.67454886e-02,
        4.54121688e-03,  9.47937518e-02, -5.53099290e-02,  1.70641169e-02,
       -2.92872749e-02, -4.47126068e-02,  2.06783861e-02,  6.39321357e-02,
        2.27427818e-02,  4.87788990e-02, -2.33504875e-03,  4.72859554e-02,
       -2.86259092e-02,  2.30624210e-02,  2.45129820e-02,  3.95681933e-02,
       -4.33177091e-02, -1.02316670e-01,  2.79879011e-03,  2.39304136e-02,
        1.61556825e-02, -8.99071433e-03,  2.07255315e-02,  6.40123338e-02,
        6.89179301e-02, -6.98362216e-02,  2.89768400e-03, -8.10988620e-02,
        1.71122216e-02,  2.50661420e-03, -1.06529057e-01, -4.87733483e-02,
       -1.67763308e-02, -

In [18]:
len(vector)

384

In [13]:
# Quick sidebar - extra to the videos - a function to compare vectors

import numpy as np
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def how_similar(text1, text2):
    vector1, vector2 = model.encode([text1, text2])
    similarity = cosine_similarity(vector1, vector2)
    print(f"Similarity between {text1} and {text2} is {similarity*100:.1f}%")

In [14]:
# And let's see how adding a few words to the context can change things up!

how_similar("Java", "C++")
how_similar("Java", "mug")
how_similar("Cup of Java", "mug")

Similarity between Java and C++ is 50.7%
Similarity between Java and mug is 25.8%
Similarity between Cup of Java and mug is 49.3%


In [15]:
# OK back to the main story - let's make something I can vectorize

def description(item):
    text = item.prompt.replace("How much does this cost to the nearest dollar?\n\n", "")
    return text.split("\n\nPrice is $")[0]

In [16]:
description(train[0])

'Delphi FG0166 Fuel Pump Module\nDelphi brings 80 years of OE Heritage into each Delphi pump, ensuring quality and fitment for each Delphi part. Part is validated, tested and matched to the right vehicle application Delphi brings 80 years of OE Heritage into each Delphi assembly, ensuring quality and fitment for each Delphi part Always be sure to check and clean fuel tank to avoid unnecessary returns Rigorous OE-testing ensures the pump can withstand extreme temperatures Brand Delphi, Fit Type Vehicle Specific Fit, Dimensions LxWxH 19.7 x 7.7 x 5.1 inches, Weight 2.2 Pounds, Auto Part Position Unknown, Operation Mode Mechanical, Manufacturer Delphi, Model FUEL PUMP, Dimensions 19.7'

## Now I populate my RAG datastore

The next cell populates the 400,000 items in Chroma.

In [17]:
NUMBER_OF_DOCUMENTS = len(train)

# Uncomment if you'd rather not wait for the full 400,000
# NUMBER_OF_DOCUMENTS = 20000

for i in tqdm(range(0, NUMBER_OF_DOCUMENTS, 1000)):
    documents = [description(item) for item in train[i: i+1000]]
    vectors = model.encode(documents).astype(float).tolist()
    metadatas = [{"category": item.category, "price": item.price} for item in train[i: i+1000]]
    ids = [f"doc_{j}" for j in range(i, i+len(documents))]
    collection.add(
        ids=ids,
        documents=documents,
        embeddings=vectors,
        metadatas=metadatas
    )

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 400/400 [1:18:17<00:00, 11.74s/it]


### Now onto Part 2b & 2c for data visualization, or skip to 2d where I construct a RAG pipeline!